In [1]:
from PIL import Image
import os, glob
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
caltech_dir = "../python_machine_learning/image2/101_ObjectCategories"

In [4]:
categories = ["wild_cat", "BACKGROUND_Google", "crayfish", "helicopter"]

In [5]:
nb_classes = len(categories)

In [6]:
X = []; Y = []
image_w = 64; image_h = 64 

In [7]:
for idx, f in enumerate(categories):
    label = [ 0 for i in range(nb_classes) ]
    label[idx] = 1

    image_dir = caltech_dir + "/" + f
    files = glob.glob(image_dir + "/*.jpg")

    for i, fname in enumerate(files):

        img = Image.open(fname)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))

        data = np.asarray(img)

        X.append(data); Y.append(label)
        
        #이미지의 각도를 변경하거나 반전 
        # Image.transpose(v) : 90도 단위로 이미지를 회전하거나 반전합니다.
        # Image.rotate(angle) : 이미지를 angle도 만큼 회전합니다.
        
        for ang in range(-20, 20, 5): 

            img2 = img.rotate(ang); data = np.asarray(img2)
            X.append(data); Y.append(label)

            # 반전하기

            img2 = img2.transpose(Image.FLIP_LEFT_RIGHT); data = np.asarray(img2)
            X.append(data); Y.append(label)


In [8]:
X = np.array(X); Y = np.array(Y)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [10]:
XY_val = (X_train, X_test, Y_train, Y_test)

In [11]:
X_train.shape

(8402, 64, 64, 3)

In [ ]:
#np.save("./object/obj_5.npy", XY_val)

## cnn 구현


In [12]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import h5py
from PIL import Image
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
categories = ["wild_cat", "BACKGROUND_Google", "crayfish", "helicopter"]

In [14]:
nb_classes = len(categories)

image_w = 64; image_h = 64

In [15]:
#X_train, X_test, Y_train, Y_test = np.load("./object/obj_5.npy")

In [16]:
X_train = X_train.astype("float") / 255; X_test = X_test.astype("float") / 255 

In [17]:
nfilter = bsize = 32; opt = ['adam','rmsprop']

In [18]:
model = Sequential()
X_train.shape, nb_classes

((8402, 64, 64, 3), 4)

In [19]:
model.add(Conv2D(nfilter, kernel_size=(3, 3), activation='relu', input_shape = X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(2*nfilter, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(2*nfilter, kernel_size=(3, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))


In [ ]:
# model.add(Conv2D(nfilter, (3, 3), padding="same", input_shape = X_train.shape[1:]))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(2*nfilter, (3, 3), padding="same"))
# model.add(Activation('relu'))
# model.add(Conv2D(2*nfilter, (3, 3)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(nb_classes))
# model.add(Activation('softmax'))

In [20]:
model.compile(loss='binary_crossentropy', optimizer=opt[1], metrics=['accuracy'])

In [21]:
hdf5_file="./object/obj_5-model.hdf5"

In [24]:
if os.path.exists(hdf5_file):
    print('load')
    model.load_weights(hdf5_file)
else:
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=bsize, epochs=5)
    model.save_weights(hdf5_file)


load


In [26]:
score = model.evaluate(X_test, Y_test, batch_size=bsize)
print("\n\n\n\nloss =", score[0], ", accuracy =", score[1])

2801/2801 [==============================] - 13s 5ms/step




loss = 0.02012353387545902 , accuracy = 0.9937522311757097
